In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 22 10:40:56 2020

@author: marcelo

ar1:
- array (nx,2), onde:
    nx: n'umero total de picos em todas as escalas
    x: 'indice do pico
    ar1[x,0]: canal inteiro do pico
    ar1[x,1]: valor da escala
    
xpk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise
    x: canal inteiro do pico
    
ypk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise (= nx(xpk))
    x: valor da escala

"""

import numpy as np
import numpy.ma as ma  # masked array
import scipy.stats as sta # statistics
from numpy import linalg as LA

from spectrum_analysis import (cwt_complete_analysis, gaussian_complete_analysis, cwt_calc_peaks)
from gamma_spectrum_class import (Spec)

from machine_selection import set_screen_size_by_machine
from bokeh.models import ColumnDataSource, Whisker, Label
from bokeh.palettes import Paired6
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import gridplot, column, row

from bokeh import palettes

# 2020-08-22 Para rodar curve_fit
from gauss_funcs import func_example, func_example_fixed_b
# 2020-09-03 Desativei por enquanto
# import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
# 2020-08-30
from scipy.interpolate import splrep, splev

from scipy.ndimage import label, generate_binary_structure, find_objects, binary_dilation  # 2020-09-03
from numpy.polynomial import Polynomial as P # 2020-09-06 Esta é a nova classe recomendada 

# 2020-09-11
from opengamma_searchfuncs import selectOverlappedPks

In [3]:
# 2020-09-12 Vou mudar hWin de 25 para 15 para testes de normalidade
hWin=15

In [6]:
nChIn = 100
nChFi = 4000
sli = slice(nChIn, nChFi)

In [7]:
x0s = spec2.xs
y0s = spec2.y0s

NameError: name 'spec2' is not defined

In [ ]:
s2k2 = np.zeros(nChPart)
skew = np.zeros(nChPart)
pvalskew = np.zeros(nChPart)
kurt = np.zeros(nChPart)
pvalkurt = np.zeros(nChPart)
pvalue = np.zeros(nChPart)
for ch in range(nChIn+hWin, nChFi-hWin):
    skew[ch], pvalskew[ch] = sta.skewtest(y0s[ch-hWin:ch+hWin+1])
    kurt[ch], pvalkurt[ch] = sta.kurtosistest(y0s[ch-hWin:ch+hWin+1])
    s2k2[ch], pvalue[ch] = sta.normaltest(y0s[ch-hWin:ch+hWin+1])

In [ ]:
# 2020-09-12 Bokeh graph
# Teste de normalidade

default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)

boka.circle(x0s[sli], y0s[sli], color='blue',size=1.5)
boka.line(x0s[sli], y0s[sli], color='green')
boka.line(x0s[sli], s2k2[sli], color='cyan')
boka.line(x0s[sli], 1.0e6 * pvalue[sli], color='orange')

# set up layout
p = gridplot([boka], ncols=1)
show(p)

In [ ]:
# 2020-09-12 Bokeh graph
# Teste de normalidade: skewness

default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)

boka.circle(x0s[sli], y0s[sli], color='blue',size=1.5)
boka.line(x0s[sli], y0s[sli], color='green')
boka.line(x0s[sli], skew[sli], color='cyan')
boka.line(x0s[sli], 1.0e6 * pvalskew[sli], color='orange')

# set up layout
p = gridplot([boka], ncols=1)
show(p)

In [ ]:
# 2020-09-12 Bokeh graph
# Teste de normalidade: kurtosis

default_plot_width, default_plot_height = set_screen_size_by_machine()
tools='pan,xwheel_zoom,xbox_select,reset'
boka = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    y_axis_type="log",
    tools=tools
)

boka.circle(x0s[sli], y0s[sli], color='blue',size=1.5)
boka.line(x0s[sli], y0s[sli], color='green')
boka.line(x0s[sli], kurt[sli], color='cyan')
boka.line(x0s[sli], 1.0e6 * pvalkurt[sli], color='orange')

# set up layout
p = gridplot([boka], ncols=1)
show(p)

In [ ]:
# 2020-08-15 AGORA interpretar os coefs. do ajuste gaussiano linearizado.

In [ ]:
# O vértice da parábola tem coord xV = -b/2a, y = c - b^2/4a.
# O fwhm será 2 delta x, onde delta x é x(H/2)-x(vertice)
# Usando a função quadrática inversa, será raiz(b^2) etc

# Os canais centroides e as alturas serão
chCtroids = np.zeros(nPks)
heights = np.zeros(nPks)
chFwhms = np.zeros(nPks)
for iPk in range(nPks):
    a = coeffs_pks[iPk,0]
    b = coeffs_pks[iPk,1]
    c = coeffs_pks[iPk,2]
    chCtroids[iPk] = -b / (2*a)
    heights[iPk] = np.exp( c - b**2 / (4*a))
    # Isso a seguir funciona mas é horrível...
    chFwhms[iPk] = np.sqrt( b**2 - 4*a*( c - np.log(heights[iPk]/2) ) ) / a
enCtroids = np.polyval(spec2.coeffs_ChEn, chCtroids)
print(chCtroids)
print(enCtroids)
print(heights)
print(chFwhms)

In [ ]:
# Perfeito!!!

In [ ]:
c

In [ ]:
2020-09-06 PAREI AQIOOOOOO!!!!!!!!

In [ ]:
# Gr'afico do ajuste:

In [ ]:
xnew_pks = np.zeros((nPks,130))
ynew_pks = np.zeros((nPks,130))
enNew_pks = np.zeros((nPks,130))
for iPk in range(nPks):
    xnew_pks[iPk] = np.linspace(pksLimits[iPk,0]+3, pksLimits[iPk,1]-3, num=130)
    ynew_pks[iPk] = np.exp(np.polyval(coeffs_pks[iPk], xnew_pks[iPk]))
    enNew_pks[iPk] = np.polyval(spec2.coeffs_ChEn, xnew_pks[iPk])

In [ ]:
xnew_pks[0]

In [ ]:
# PAREI AQUI a parte das paletas gr'aficas  2020-ago-12

In [ ]:
Paired6

In [ ]:
Pretos = ('#a6cee3', '#1f78b4', '#b2df8a', '#a6cee3', '#a6cee3', '#a6cee3')

In [ ]:
base, lower, upper = [], [], []

for i, x in enumerate(list(spec2.xs)):
    lower.append(spec2.y0s[i]-spec2.uncCounts[i])
    upper.append(spec2.y0s[i]+spec2.uncCounts[i])
    base.append(x)

source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

# Para nao cortar o numero da escala y:
yrange [1] = 1.5e6

bok1z = figure(
    plot_width = 900,
    # plot_width =default_plot_width,
    plot_height=default_plot_height,
    tools="pan,box_zoom,reset,save",
    y_axis_type="log",
    x_range=[22,39],
    y_range=yrange,
#    title=fn2
)
# bok1z.title.text = 'Click on legend entries to hide the corresponding lines'
bok1z.xaxis.axis_label = 'Energy (keV)'
bok1z.xaxis.axis_label_text_font_size = '18pt'
bok1z.xaxis.major_label_text_font_size = '16pt'

bok1z.yaxis.axis_label = 'Counts'
bok1z.yaxis.axis_label_text_font_size = '18pt'
bok1z.yaxis.major_label_text_font_size = '16pt'

base, lower, upper = [], [], []
for i, x in enumerate(list(ens)):
    lower.append(ys[i]-uncys[i])
    upper.append(ys[i]+uncys[i])
    base.append(x)
source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

bok1z.circle(ens, ys, size=3, color='navy', alpha=1.0, legend_label='ys')
bok1z.line(ensnew, ynew, line_width=0.7, color='red', alpha=0.9, legend_label='gross')
bok1z.line(ens, contin, line_width=0.4, color='green', alpha=0.8, legend_label='contin')
bok1z.square(ens, yNets, size=7, line_color='navy', fill_color='orange', alpha=0.6, legend_label='net')
bok1z.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
) 

bok1z.legend.location = "top_right"
bok1z.legend.click_policy="hide"

bok1z.multi_line(list(enNew_pks), list(ynew_pks), line_width=0.3, color='blue', alpha=0.9, legend_label='pks')

mytext = Label(x=27.7, y=6.0e5, text='27 keV', text_font_size='18pt')
bok1z.add_layout(mytext)

mytext = Label(x=31.3, y=1.3e5, text='31 keV', text_font_size='18pt')
bok1z.add_layout(mytext)

mytext = Label(x=35.8, y=4.0e4, text='35 keV', text_font_size='18pt')
bok1z.add_layout(mytext)

show(bok1z)

In [ ]:
enNew_pks.shape

In [ ]:
# 2020-07-11 Bokeh graph w/ 2nd grade fit coefficients

uncys = np.sqrt(ys)

bokzz = figure(
    plot_width =default_plot_width,
    plot_height=default_plot_height,
    tools="pan,box_zoom,reset,save",
    y_axis_type="log",
    x_range=[20,40],
    y_range=yrange,
)

bokzz.xaxis.axis_label = 'Energy (keV)'
bokzz.xaxis.axis_label_text_font_size = '16pt'
bokzz.xaxis.major_label_text_font_size = '14pt'

bokzz.yaxis.axis_label = 'Counts'
bokzz.yaxis.axis_label_text_font_size = '16pt'
bokzz.yaxis.major_label_text_font_size = '14pt'

base, lower, upper = [], [], []

for i, x in enumerate(list(ens)):
    lower.append(ys[i]-uncys[i])
    upper.append(ys[i]+uncys[i])
    base.append(x)

source_error = ColumnDataSource(data=dict(base=base, lower=lower, upper=upper))

bokzz.add_layout(
    Whisker(source=source_error, base="base", upper="upper", lower="lower")
)    
    
bokzz.circle(spec2.xs, spec2.y0s, color="red",size=4)
bokzz.line(spec2.xs, spec2.y0s)

show(bokzz)

In [ ]:
import pyplot as plt

In [ ]:
xdata = np.linspace(0, 4, 50)
y = func_example(xdata, 2.5, 1.3, 0.5)
np.random.seed(1729)
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='data')
popt, pcov = curve_fit(func_example, xdata, ydata)
popt

plt.plot(xdata, func_example(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
#popt, pcov = curve_fit(func_example, xdata, ydata, bounds=(0, [3., 1., 0.5]))
popt, pcov = curve_fit(func_example, xdata, ydata, bounds=([3., 1., 0.5], [3.0001, 1.0001, 0.5001]))
popt

plt.plot(xdata, func_example(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
xdata = np.linspace(0, 4, 50)
y = func_example(xdata, 2.5, 1.3, 0.5)
np.random.seed(1729)
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='data')
popt, pcov = curve_fit(func_example, xdata, ydata)
popt

plt.plot(xdata, func_example(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
popt, pcov = curve_fit(func_example, xdata, ydata, bounds=(0, [3., 1., 0.5]))
# popt, pcov = curve_fit(func_example, xdata, ydata, bounds=([3., 1., 0.5], [3.0001, 1.0001, 0.5001]))
popt

plt.plot(xdata, func_example(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
xdata = np.linspace(0, 4, 50)
y = func_example(xdata, 2.5, 1.3, 0.5)
np.random.seed(1729)
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='data')
popt, pcov = curve_fit(func_example, xdata, ydata)
popt

plt.plot(xdata, func_example(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
popt, pcov = curve_fit(func_example, xdata, ydata, bounds=(0, [3., 1., 0.5]))
# popt, pcov = curve_fit(func_example, xdata, ydata, bounds=([3., 1., 0.5], [3.0001, 1.0001, 0.5001]))
popt

plt.plot(xdata, func_example(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

In [ ]:
fixed_b = 1.313

In [ ]:
xdata2 = np.concatenate((xdata, np.array([fixed_b])))

In [ ]:
# xdata = np.linspace(0, 4, 50)
# y = func_example(xdata, 2.5, 1.3, 0.5)
# np.random.seed(1729)
# y_noise = 0.2 * np.random.normal(size=xdata.size)
# ydata = y + y_noise

popt, pcov = curve_fit(func_example_fixed_b, xdata2, ydata)
tupla = (popt[0], popt[1], fixed_b) # QUELLE HORREUR

plt.plot(xdata, func_example_fixed_b(xdata2, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tupla)

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()
print(popt)